In [1]:
import glob
import pickle
import numpy
from music21 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf


tf.compat.v1.disable_eager_execution()

In [2]:
#Extraction of all notes in sequence from the training songs
notes = []
song_count=0
for file in glob.glob("training_songs/*.mid"):
  song_count = song_count + 1
  midi = converter.parse(file)

  print("Parsing %s" % file)

  notes_to_parse = None

  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    #If notes
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    #If chords
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))

print("\nTotal Songs = ",song_count)      

Parsing training_songs/balamb.mid
Parsing training_songs/ff4_piano_collections-main_theme.mid
Parsing training_songs/Eternal_Harvest.mid
Parsing training_songs/traitor.mid
Parsing training_songs/Finalfantasy5gilgameshp.mid
Parsing training_songs/decisive.mid
Parsing training_songs/tpirtsd-piano.mid
Parsing training_songs/FFIX_Piano.mid
Parsing training_songs/bcm.mid
Parsing training_songs/Finalfantasy6fanfarecomplete.mid
Parsing training_songs/rufus.mid
Parsing training_songs/FF4.mid
Parsing training_songs/fortresscondor.mid
Parsing training_songs/Rydia_pc.mid
Parsing training_songs/Rachel_Piano_tempofix.mid
Parsing training_songs/In_Zanarkand.mid
Parsing training_songs/Fyw_piano.mid
Parsing training_songs/electric_de_chocobo.mid
Parsing training_songs/pkelite4.mid
Parsing training_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing training_songs/ff6shap.mid
Parsing training_songs/FF6epitaph_piano.mid
Parsing training_songs/ff11_awakening_piano.mid
Parsing training_songs/HighwindTakes

In [3]:
print(notes[0:20])

['F3', 'C4', '4.9', 'E4', 'C4', '2.5', 'E4', 'C4', '9.0', 'E4', 'C4', 'F3', 'C4', '4.9', 'E4', 'C4', '2.5', 'E4', 'C4', '9.0']


In [4]:
n_class = len(set(notes))
print(n_class)

358


In [5]:
#Generating training sequences from the extracted notes


sequence_length = 100

#Get all pitch names
pitchnames = sorted(set(item for item in notes))

#Create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

#Create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

#Reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
#Normalize input
network_input = network_input / float(n_class)

network_output = to_categorical(network_output)

In [6]:
print(network_input.shape)

(57077, 100, 1)


In [7]:
#Creating the model structure
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(n_class))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam')

In [ ]:
#You can skip the training as it can take a long time and load the weights directly in the next cell
model.fit(network_input, network_output, batch_size=128, epochs=200)
model.save_weights("model.hdf5")

In [8]:
#Load Weigts
model.load_weights('model.hdf5')

In [9]:
#Pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)
print("Sequence_Index = ", start)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

#Generate 500 notes
for note_index in range(500):
    if note_index%10==0:
        print("Generating Note: ",note_index)
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    

    prediction = model.predict(prediction_input)


    idx = numpy.argmax(prediction)
    result = int_to_note[idx]
    prediction_output.append(result)

    
    pattern = pattern[1:]
    idx = idx/float(n_class)
    pattern = numpy.append(pattern, idx)


Sequence_Index =  33117
Generating Note:  0


/home/aneeshak/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Generating Note:  10
Generating Note:  20
Generating Note:  30
Generating Note:  40
Generating Note:  50
Generating Note:  60
Generating Note:  70
Generating Note:  80
Generating Note:  90
Generating Note:  100
Generating Note:  110
Generating Note:  120
Generating Note:  130
Generating Note:  140
Generating Note:  150
Generating Note:  160
Generating Note:  170
Generating Note:  180
Generating Note:  190
Generating Note:  200
Generating Note:  210
Generating Note:  220
Generating Note:  230
Generating Note:  240
Generating Note:  250
Generating Note:  260
Generating Note:  270
Generating Note:  280
Generating Note:  290
Generating Note:  300
Generating Note:  310
Generating Note:  320
Generating Note:  330
Generating Note:  340
Generating Note:  350
Generating Note:  360
Generating Note:  370
Generating Note:  380
Generating Note:  390
Generating Note:  400
Generating Note:  410
Generating Note:  420
Generating Note:  430
Generating Note:  440
Generating Note:  450
Generating Note:  4

In [10]:
#Process the output notes and create a midi file
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='generated_samples/test_output2.mid')


'generated_samples/test_output2.mid'